## Data Loading



In [1]:
import pandas as pd

# 加载播放记录（用户-歌曲-播放次数）
triplets = pd.read_csv('train_triplets.txt', sep='\t', header=None, names=['user_id', 'song_id', 'plays'])
print(triplets.head())

                                    user_id             song_id  plays
0  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOAKIMP12A8C130995      1
1  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOAPDEY12A81C210A9      1
2  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBBMDR12A8C13253B      2
3  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBFNSP12AF72A0E22      1
4  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBFOVM12A58A7D494      1


In [3]:
import pandas as pd
import h5py

def load_metadata(filename):
    with h5py.File(filename, "r") as f:
        songs_dataset = f['metadata']['songs']
        
        # 提取原始字节数据
        song_ids_bytes = songs_dataset['song_id'][()]  # 字节数组
        titles_bytes = songs_dataset['title'][()]      # 字节数组
        
        # 安全解码为 UTF-8（处理非法字符）
        song_ids = [s.decode('utf-8', errors='ignore').strip() for s in song_ids_bytes]
        titles = [t.decode('utf-8', errors='ignore').strip() for t in titles_bytes]
        
        # 构建 DataFrame
        df = pd.DataFrame({
            'song_id': song_ids,
            'title': titles
        })
        
        # 移除空 song_id
        df = df[df['song_id'].str.len() > 0]
    return df

metadata = load_metadata('msd_summary_file.h5')

# 合并数据
merged_data = pd.merge(
    triplets,
    metadata,
    on='song_id',
    how='left'
)

# 验证结果
print("标题缺失比例:", merged_data['title'].isnull().mean())
print("示例数据:")
print(merged_data.head())

标题缺失比例: 0.0
示例数据:
                                    user_id             song_id  plays  \
0  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOAKIMP12A8C130995      1   
1  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOAPDEY12A81C210A9      1   
2  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBBMDR12A8C13253B      2   
3  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBFNSP12AF72A0E22      1   
4  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBFOVM12A58A7D494      1   

                             title  
0                         The Cove  
1             Nothing from Nothing  
2                  Entre Dos Aguas  
3            Under Cold Blue Stars  
4  Riot Radio (Soundtrack Version)  


## Training

In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Multiply, Dense
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
import joblib
from pathlib import Path
from tensorflow.keras.layers import (
    Input, Embedding, Flatten, 
    Concatenate, Dropout, Dense  
)

from tensorflow.keras.layers import Concatenate

# ----------------------
# 数据预处理
# ----------------------

# 1. 用户和歌曲ID编码（转换为连续整数）
user_encoder = LabelEncoder()
song_encoder = LabelEncoder()

# 对用户ID和歌曲ID进行编码
merged_data['user_id_encoded'] = user_encoder.fit_transform(merged_data['user_id'])
merged_data['song_id_encoded'] = song_encoder.fit_transform(merged_data['song_id'])

# 2. 归一化播放次数到 [0,1]
max_play = merged_data['plays'].max()
merged_data['plays_normalized'] = merged_data['plays'] / max_play

# 3. 提取训练数据
user_ids = merged_data['user_id_encoded'].values
item_ids = merged_data['song_id_encoded'].values
labels = merged_data['plays_normalized'].values  # 归一化后的播放次数

# 4. 划分训练集和测试集
train_user, test_user, train_item, test_item, train_label, test_label = train_test_split(
    user_ids, item_ids, labels, test_size=0.2, random_state=42
)

# ----------------------
# 模型构建
# ----------------------

# 定义用户和物品数量
num_users = len(user_encoder.classes_)  # 105,283
num_items = len(song_encoder.classes_)  # 384,546
embedding_size = 32  # 嵌入维度



tf.config.threading.set_inter_op_parallelism_threads(4)
tf.config.threading.set_intra_op_parallelism_threads(4)
def build_fusion_model(gmf_model=None, mlp_model=None):
    pretrained_path = "best_fusion_model.keras"
    
    if Path(pretrained_path).exists():
        print("⏳ 检测到预训练Fusion模型，加载中...")
        model = tf.keras.models.load_model(pretrained_path)
        print("✅ 成功加载预训练模型")
        return model
    
    print("🆕 创建Fusion融合模型")
    user_input = Input(shape=(1,), name='user_input')
    item_input = Input(shape=(1,), name='item_input')

    def safe_load_submodel(path, prefix):
        model = tf.keras.models.load_model(path)
        for layer in model.layers:
            layer._name = f"{prefix}_{layer.name}"
            if isinstance(layer, tf.keras.Model):
                for sublayer in layer.layers:
                    sublayer._name = f"{prefix}_{sublayer.name}"
        return model

    # GMF分支
    if gmf_model and Path(gmf_model).exists():
        print("💡 加载预训练GMF组件")
        # 1. 安全加载子模型并统一加前缀
        gmf_submodel = safe_load_submodel(gmf_model, "gmf")
        # 2. 提取预训练的 Embedding 权重
        gmf_user_weights = gmf_submodel.get_layer("gmf_user_embed").get_weights()
        gmf_item_weights = gmf_submodel.get_layer("gmf_item_embed").get_weights()
        # 3. 用提取到的权重构建融合用 Embedding，并加载权重
        fusion_gmf_user_embed = Embedding(
            num_users, 16,
            name="fusion_gmf_user_embed",
            
            weights=gmf_user_weights,
            trainable=True
        )(user_input)
        fusion_gmf_item_embed = Embedding(
            num_items, 16,
            name="fusion_gmf_item_embed",
            weights=gmf_item_weights,
            trainable=True
        )(item_input)
        # 4. 逐元素相乘 + 拉平
        fusion_gmf_mul     = Multiply(name="fusion_gmf_mul")([fusion_gmf_user_embed, fusion_gmf_item_embed])
        gmf_flatten        = Flatten(name="fusion_gmf_flatten")(fusion_gmf_mul)
    else:
        print("💡 初始化新GMF分支")
        gmf_user_embed = Embedding(num_users, 16, name='gmf_user_embed')(user_input)
        gmf_item_embed = Embedding(num_items, 16, name='gmf_item_embed')(item_input)
        gmf_output = Multiply()([Flatten()(gmf_user_embed), Flatten()(gmf_item_embed)])
        gmf_flatten = Flatten(name='gmf_flatten')(gmf_output)
    # MLP分支
    if mlp_model and Path(mlp_model).exists():
        print("💡 加载预训练MLP组件")
        # 1. 安全加载子模型并统一加前缀
        mlp_submodel = safe_load_submodel(mlp_model, "mlp")
        # 2. 提取预训练的 Embedding 权重
        mlp_user_weights = mlp_submodel.get_layer("mlp_user_embed").get_weights()
        mlp_item_weights = mlp_submodel.get_layer("mlp_item_embed").get_weights()
        # 3. 用提取到的权重构建融合用 Embedding，并加载权重
        fusion_mlp_user_embed      = Embedding(
            num_users, 64,
            name="fusion_mlp_user_embed",
            weights=mlp_user_weights,
            trainable=True
        )(user_input)
        fusion_mlp_item_embed      = Embedding(
            num_items, 64,
            name="fusion_mlp_item_embed",
            weights=mlp_item_weights,
            trainable=True
        )(item_input)
        # 4. 展平再拼接
        fusion_mlp_user_flatten    = Flatten(name="fusion_mlp_user_flatten")(fusion_mlp_user_embed)
        fusion_mlp_item_flatten    = Flatten(name="fusion_mlp_item_flatten")(fusion_mlp_item_embed)
        mlp_concat                 = Concatenate(name="fusion_mlp_concat")(
            [fusion_mlp_user_flatten, fusion_mlp_item_flatten]
        )
        # 5. 新的 MLP 隐藏层
        fusion_mlp_dense1          = Dense(
            256, activation="relu", name="fusion_mlp_dense1"
        )(mlp_concat)
        fusion_mlp_dropout         = Dropout(0.2, name="fusion_mlp_dropout")(fusion_mlp_dense1)
        fusion_mlp_dense2          = Dense(
            128, activation="relu", name="fusion_mlp_dense2"
        )(fusion_mlp_dropout)
        mlp_output                 = Dense(
            64, activation="relu", name="fusion_mlp_output"
        )(fusion_mlp_dense2)
    else:
        print("💡 初始化新MLP分支")
        mlp_user_embed = Embedding(num_users, 64, name='mlp_user_embed')(user_input)
        mlp_item_embed = Embedding(num_items, 64, name='mlp_item_embed')(item_input)
        mlp_concat = Concatenate()([Flatten()(mlp_user_embed), Flatten()(mlp_item_embed)])
        mlp_dense = Dense(256, activation='relu')(mlp_concat)
        mlp_dense = Dropout(0.2)(mlp_dense)
        mlp_dense = Dense(128, activation='relu')(mlp_dense)
        mlp_output = Dense(64, activation='relu')(mlp_dense)

    # 融合层
    merged = Concatenate(name='fusion_concat')([gmf_flatten, mlp_output])
    final_dense = Dense(32, activation='relu', name='fusion_dense')(merged)
    output = Dense(1, activation='linear', name='fusion_output')(final_dense)

    model = Model(inputs=[user_input, item_input], outputs=output)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae'],
        steps_per_execution=50,  # 每次 tf.function 调用跑 50 个 batch
    )
    return model
# ----------------------
# 初始化并训练融合模型
# ----------------------
# 加载预训练组件（可选）
model = build_fusion_model(
    gmf_model="best_gmf_model.keras",
    mlp_model="best_mlp_model.keras"
)
model.summary()


batch_size = 16384

train_dataset = tf.data.Dataset.from_tensor_slices(
    ((train_user, train_item), train_label)
).cache().shuffle(100000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices(
    ((test_user, test_item), test_label)
).cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)



# 添加模型保存回调（自动保存最佳模型）
class CustomCheckpoint(ModelCheckpoint):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # 添加优化器状态保存路径
        self.optimizer_path = "optimizer_state_fusion.pkl"
    
    def on_train_end(self, logs=None):
        # 保存优化器权重
        joblib.dump(self.model.optimizer.get_weights(), self.optimizer_path)
        print(f"💾 已保存优化器状态至 {self.optimizer_path}")

# 更新检查点路径

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    verbose=1,
    restore_best_weights=True   # 是否恢复到最佳权重
)

checkpoint = CustomCheckpoint(
    "best_fusion_model.keras",  # 修改保存文件名
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

# 如果检测到优化器状态则加载
if Path("optimizer_state_fusion.pkl").exists():
    print("⏳ 加载优化器状态...")
    optimizer_weights = joblib.load("optimizer_state_fusion.pkl")
    model.optimizer.set_weights(optimizer_weights)
    print("✅ 优化器状态已恢复")

# 训练模型（epochs可根据需要调整）
history = model.fit(
    [train_user, train_item],             # 两个 numpy array
    train_label,                          # 标签
    batch_size=16384,                     # 根据显存可增大/减小
    epochs=10,
    validation_data=([test_user, test_item], test_label),
    callbacks=[early_stop, checkpoint],
    verbose=1                             # batch 级进度条
)
# ----------------------
# 模型评估与预测
# ----------------------

# 评估测试集
test_loss, test_mae = model.evaluate(test_dataset)
print(f"testsets MSE: {test_loss:.4f}, MAE: {test_mae:.4f}")

# 保存编码器（训练后立即执行）
joblib.dump(max_play, 'max_play_fusion.pkl')
joblib.dump(user_encoder, 'user_encoder_fusion.pkl')
joblib.dump(song_encoder, 'song_encoder_fusion.pkl')


🆕 创建Fusion融合模型
💡 加载预训练GMF组件
💡 加载预训练MLP组件
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
fusion_mlp_user_embed (Embeddin (None, 1, 64)        65236352    user_input[0][0]                 
__________________________________________________________________________________________________
fusion_mlp_item_embed (Embeddin (None, 1, 64)        24610944    item_input[0][0]                 
_____________________________________________________

c:\Users\Piper\anaconda3\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/10
2426/2426 [==============================] - 74s 31ms/step - loss: 4.2696e-07 - mae: 2.5634e-04 - val_loss: 5.0945e-07 - val_mae: 2.4500e-04

Epoch 00002: val_loss improved from 0.00000 to 0.00000, saving model to best_fusion_model.keras
Epoch 3/10
2426/2426 [==============================] - 75s 31ms/step - loss: 4.2092e-07 - mae: 2.5702e-04 - val_loss: 5.4190e-07 - val_mae: 2.5097e-04

Epoch 00003: val_loss did not improve from 0.00000
Epoch 4/10
2426/2426 [==============================] - 75s 31ms/step - loss: 4.2384e-07 - mae: 2.5995e-04 - val_loss: 5.3875e-07 - val_mae: 2.6131e-04

Epoch 00004: val_loss did not improve from 0.00000
Epoch 5/10
2426/2426 [==============================] - 95s 39ms/step - loss: 4.2399e-07 - mae: 2.5658e-04 - val_loss: 5.3580e-07 - val_mae: 2.6433e-04
Restoring model weights from the end of the best epoch.

Epoch 00005: val_loss did not improve from 0.00000
Epoch 00005: early stopping
💾 已保存优化器状态至 optimizer_state_fusion.pkl
500/607 [=======